In [ ]:
############################################################################################
# IMPORTS
############################################################################################
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#import tensorflow.keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
#import tensorflow.keras as keras

from tensorflow.keras import layers
from tensorflow.keras import regularizers


from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Activation, Flatten

from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical



from IPython import display
from PIL import Image


import pathlib
import shutil
import tempfile
import concurrent


import json
import glob





print(tf.__version__)
print(tf.config.list_physical_devices())
############################################################################################

In [ ]:
############################################################################################
# 1. CONSTANTS - PATHS
############################################################################################

DATA_FS251 = './data/iFood_2019'
CLASSES_FILE_NAME = '/formated_annot/classes_formated.csv'

TRAIN_INFO = '/annot/train_info.csv'
VAL_INFO = '/annot/val_info.csv'
TEST_INFO = '/annot/test_info.csv'

TRAIN_PICS_PATH = './data/iFood_2019/train_set/'
TEST_PICS_PATH = './data/iFood_2019/test_set/'
VAL_PICS_PATH = './data/iFood_2019/val_set/'

SEED = 111

#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


In [ ]:
############################################################################################
# IMPORTING DATA
############################################################################################
df_classes = pd.read_csv(DATA_FS251 + CLASSES_FILE_NAME)
df_train = pd.read_csv(DATA_FS251 + TRAIN_INFO, names=['file_name', 'class_num'])
df_validate = pd.read_csv(DATA_FS251 + VAL_INFO, names=['file_name', 'class_num'])
df_test = pd.read_csv(DATA_FS251 + TEST_INFO, names=['file_name'])

df_train.head(3)

In [ ]:
############################################################################################
# 2. CONSTANTS - MODEL
############################################################################################
training_history = dict()

N_TRAIN = len(df_train.iloc[:, 0])
EPOCHS = 20
BATCH_SIZE = 8
RESIZE_TO = (224, 224)
STEPS_PER_EPOCH = N_TRAIN // BATCH_SIZE
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
    0.01,
    decay_steps=STEPS_PER_EPOCH*1000,
    decay_rate=1,
    staircase=False)

In [ ]:
############################################################################################
# import image files

train_pics = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_PICS_PATH,
    labels='inferred',
    label_mode='categorical',
    class_names=None,
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=RESIZE_TO,
    shuffle=False,
    seed=SEED,
    validation_split=None,
    subset=None,
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=True,
    data_format=None,
    verbose=True
)
val_pics = tf.keras.preprocessing.image_dataset_from_directory(
    VAL_PICS_PATH,
    labels='inferred',
    label_mode='categorical',
    class_names=None,
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=RESIZE_TO,
    shuffle=False,
    seed=SEED,
    validation_split=None,
    subset=None,
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=True,
    data_format=None,
    verbose=True
)
test_pics = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_PICS_PATH,
    labels=None,
    label_mode=None,
    class_names=None,
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=RESIZE_TO,
    shuffle=False,
    seed=SEED,
    validation_split=None,
    subset=None,
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=True,
    data_format=None,
    verbose=True
)

In [ ]:
############################################################################################
# VGG16 model construction/deffinition
############################################################################################
tf.keras.backend.clear_session()
model_supclass_vgg16 = tf.keras.models.Sequential([
        tf.keras.layers.Resizing(
                height = 224,
                width = 224,
                interpolation='bilinear',
                crop_to_aspect_ratio=False,
                pad_to_aspect_ratio=False,
                fill_mode='constant',
                fill_value=0.0,
                data_format=None),
        tf.keras.applications.vgg16.VGG16(weights='imagenet'),
        tf.keras.layers.Dense(
                units = 251,
                activation = tf.keras.activations.sigmoid)
])

In [ ]:
############################################################################################
# COMPILING
############################################################################################

lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
    0.3,
    decay_steps=STEPS_PER_EPOCH*1000,
    decay_rate=1,
    staircase=False)
model_supclass_vgg16.compile(
              optimizer = tf.keras.optimizers.Adam(learning_rate = lr_schedule),
              loss = tf.keras.losses.categorical_crossentropy,
              metrics = [tf.keras.metrics.categorical_accuracy] 
)
model_supclass_vgg16.summary()

In [ ]:
############################################################################################
# FITTING
############################################################################################
with tf.device('/gpu:0'):
    model_history = model_supclass_vgg16.fit(
        train_pics,
        epochs = EPOCHS, 
        batch_size = BATCH_SIZE, 
        validation_data = val_pics)

model_supclass_vgg16.summary()

In [ ]:
STOP_HERE

In [ ]:
############################################################################################
# EVALUATION
model_history.history.keys()
pd.DataFrame(model_history.history).plot()
plt.legend(bbox_to_anchor = [1, 1.02])
plt.plot(model_history.history['val_loss'])
plt.title('Validation Loss')
plt.xlabel('epochs')

# Print out the score
score = model.evaluate(X, y , batch_size=10, verbose=0)
print(score, model.metrics_names)

############################################################################################
# PREDICTION








y_hat=model.predict(new_X)
############################################################################################
# SAVING MODEL




model.save("model_moons.h5")
moons_model = load_model("model_moons.h5")

